<a href="https://colab.research.google.com/github/hyejwon/1_ne/blob/main/%EB%AA%A8%EB%8D%B8%EC%A0%80%EC%9E%A5%EA%B3%BC_%EB%B3%B5%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 모델 저장과 복원  

훈련하는 도중이나 훈련이 끝난 후에 모델을 저장할 수 있습니다. 모델을 중지된 지점부터 다시 훈련할 수 있어 한 번에 오랫동안 훈련하지 않아도 됩니다. 또 모델을 저장하면 다른 사람에게 공유할 수 있고 작업을 재현할 수 있습니다. 연구한 모델과 기법을 공개할 때 많은 머신 러닝 기술자들이 다음과 같은 것들을 제공합니다:

모델을 만드는 코드  
모델의 훈련된 가중치 또는 파라미터
이런 데이터를 공유하면 다른 사람들이 모델의 작동 방식을 이해하고 새로운 데이터로 모델을 실험하는데 도움이 됩니다.

주의: 신뢰할 수 없는 코드는 조심하세요. 텐서플로 모델은 프로그램 코드입니다. 자세한 내용은 텐서플로를 안전하게 사용하기 문서를 참고하세요.

저장 방식  
사용하는 API에 따라서 여러가지 방법으로 텐서플로 모델을 저장할 수 있습니다. 이 문서는 텐서플로 모델을 만들고 훈련하기 위한 고수준 API인 tf.keras를 사용합니다. 다른 방법들에 대해서는 텐서플로의 저장과 복원 문서와 즉시 실행(eager execution) 문서의 저장하기 섹션을 참고하세요.


### 설정
#### 설치와 임포트


In [ ]:
import os 
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

In [ ]:
!pip install pyyaml h5py

In [ ]:
### 예제 데이터 셋 불러오기
(train_images,train_labels),(test_images,test_labels)=tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000] #처음 1,000개만 사용
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = train_images[:1000].reshape(-1,28*28)/255.0 #flatten


### 모델 정의


In [ ]:
#간단한 Sequential 모델

def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

#모델 객체를 만든다
model = create_model()

#모델 구조를 출력
model.summary()


## 훈련하는 동안 체크포인트 저장하기
훈련 중간과 훈련 마지막에 체크포인트를 자동으로 저장하도록 하는 것이 많이 사용하는 방법.  
다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있다. ```tf.keras.callbacks.ModelCheckpoint``` 은 이런 작업을 수행하는 콜백이다. 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공  
### 체크포인트 콜백 사용하기
```ModelCheckpoint``` 만들기

이 코드는 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때 마다 업데이트 

In [ ]:
checkpoint_path="training_1/cp.ckpt"
checkpoint_dir=os.path.dirname(checkpoint_path)

#모델의 가중치를 저장하는 콜백 만들기 
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

#새로운 콜백으로 모델 훈련하기
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback]) #콜백을 훈련에 전달

#옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있다.
#이 경고는(그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋다


In [ ]:
!ls {checkpoint_dir}

훈련하지 않은 새로운 모델을 만들기. 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 한다. 여기서는 동일한 구조로 모델을 만들었으므로 다른 객체이지만 가중치를 공유할 수있다.  
훈련하지 않은 새 모델을 만들고 테스트 세트에서 평가. 훈련되지 않은 모델의 성능은 무작위로 선택하는 정도의 수준(~10% 정확도)

In [ ]:
#기본 모델 객체를 만든다
model = create_model()

#모델을 평가
loss, acc = model.evaluate(test_images,test_labels,verbose=2)
print("훈련되지 않은 모델의 정확도:{:5.2f}%".format(100*acc))

체크포인트에서 가중치를 로드하고 다시 평가

In [ ]:
#가중치 로드 
model.load_weights(checkpoint_path)

#모델 재평가
loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도:{:5.2f}%".format(100*acc))

### 체크포인트 콜백 매개변수
이 콜백 함수는 몇 가지 매개변수를 제공 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정  

새로운 모델을 훈련하고 다섯 번의 에포크마다 고유한 이름으로 체크포인트를 저장

In [ ]:
#파일 이름에 에포크 번호를 포함시킨다('str.format'포맷)
checkpoint_path="training_2/cp-{epoch:04d}.ckpt"#포매팅 :십진수 정수 4자리 표현 나머지 자리는 0으로 메꿈
checkpoint_dir= os.path.dirname(checkpoint_path)

#다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만든다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

#새로운 모델 객체를 만든다.
model = create_model()

#'checkpoint_path' 포맷을 사용하는 가중치를 저장한다
model.save_weights(checkpoint_path.format(epoch=0))

#새로운 콜백을 사용하여 모델을 훈련
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)

In [ ]:
ls {checkpoint_dir}

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

텐서플로는 기본적으로 최근 5개의 체크포인트만 저장한다.  
모델을 초기화하고 최근 체크포인트를 로드하여 테스트 

In [ ]:
#새로운 모델 객체를 만든다.
model = create_model()

#이전에 저장한 가중치를 로드한다
model.load_weights(latest)

#모델을 재평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도 : {:5.2f}%".format(100*acc))

### 수동으로 가중치 저장하기  
앞에서 가중치를 모델에 로드하는 방법을 보았다. 수동으로 가중치를 저장하는 것도 쉽다.
```Model.save_weights``` 메서드를 사용한다. ```tf.keras``` 는, 특히 ```save_weights```는 .ckpt확장자를 가진 텐서플로 체크포인트 포맷을 사용

In [ ]:
#가중치를 저장
model.save_weights('./checkpoints/my_checkpoint')

#새로운 모델 객체를 만듦
model = create_model()

#가중치를 복원
model.load_weights('./checkpoints/my_checkpoint')

#모델 평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도:{:5.2f}%".format(100*acc))

## 전체 모델 저장하기
model.save 메서드를 호출하여 모델의 구조,가중치,훈련 설정을 하나의 파일/폴더에 저장한다. 모델을 저장하기 때문에 원본 파이썬 코드 * 가 없어도 사용할수 있다. 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 다시 훈련을 시작할 수 있다.  

두개의 포맷(SavedModel과 HDF5)으로 모델을 저장할 수 있다. 텐서플로의 SavedModel 포맷은 TF2.x에서 기본 파일 포맷이다. 하지만 HDF5 포맷으로 저장할 수도 있다. 두 파일 포맷으로 전체 모델을 저장하는 방법은 아래에서 자세히 설명  

전체 모델을 저장하는 기능은 매우 유용하다. TensorFlow.js 로 모델을 로드한 다음 웹브라우저에서 모델을 훈련하고 실행할 수 있다.(Saved Model,HDF5). 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite 를 사용하여 실행할 수 있다.(Saved Model, HDF5)  

*사용자 정의 객체(예를 들면 상속으로 만든 클래스나 층)는 저장하고 로드하는데 특별한 주의가 필요

### SavedModel 포맷

SavedModel 포맷을 모델을 직렬화 하는 다른 방법이다. 이 포맷으로 저장한 모델은 ```tf.keras.models.load_model``` 로 복원할 수 있고 텐서플로 서빙과 호환된다. SavedModel 가이드에서 SavedModel를 서빙하고 점검하는 자세한 방법을 제공 이 섹션에서는 모델을 저장하고 복원하는 방법을 안내

In [ ]:
#새로운 모델 객체를 만들고 훈련
model = create_model()
model.fit(train_images, train_labels, epochs=5)

#SavedModel로 전체 모델을 저장
!mkdir - p saved_model
model.save('saved_model/my_model')

In [ ]:
#SavedModel 포맷은 프로토콜 버퍼 이진파일과 텐서플로 체크포인트를 담고있는 디렉토리
#my model 디렉토리
!ls saved_model

#assets 폴더, saved_model.pb, variables 폴더
!ls saved_model/my_model

In [ ]:
#저장된 모델로부터 새로운 케라스 모델을 로드

new_model = tf.keras.models.load_model('saved_model/my_model')

#모델 구조를 확인
new_model.summary()

복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있음. 이 모델을 평가하고 예측에 사용

In [ ]:
#복원된 모델을 평가
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

### HDF5 파일로 저장하기 

In [ ]:
#새로운 모델 객체를 만들고 훈련
model = create_model()
model.fit(train_images,train_labels,epochs=5)

#전체 모델을 HDF5 파일로 저장한다
#'.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타낸다

model.save('my_model.h5')

In [ ]:
#가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성

new_model = tf.keras.models.load_model('my_model.h5')

#모델 구조를 출력한다
new_model.summary()

정확도를 확인

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('복원된 모델의 정확도:{:5.2f}%'.format(100*acc))
